# 完整的训练套路

In [16]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential, CrossEntropyLoss


# 获取数据集
train_data = torchvision.datasets.CIFAR10(root="./tv_dataset", train=True, transform=ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root="./tv_dataset", train=False, transform=ToTensor(), download=True)
train_data_size = len(train_data)
test_data_size = len(test_data)
print("训练集长度 : {}, 测试集长度 : {}".format(train_data_size, test_data_size))

# 使用dataloader加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


Files already downloaded and verified
Files already downloaded and verified
训练集长度 : 50000, 测试集长度 : 10000


In [17]:
# 搭建神经网络
class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.model1 = Sequential(
            Conv2d(3, 32, kernel_size=5, padding=2),
            MaxPool2d(kernel_size=2),
            Conv2d(32, 32, kernel_size=5, padding=2),
            MaxPool2d(kernel_size=2),
            Conv2d(32, 64, kernel_size=5, padding=2),
            MaxPool2d(kernel_size=2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )
        
    def forward(self, x):
        x = self.model1(x)
        return x

In [22]:
# 创建网络模型
tudui = Tudui()

# 损失函数
loss_fn = CrossEntropyLoss()

# 定义优化器
learning_rate = 1e-2
optim = torch.optim.SGD(tudui.parameters(), lr=learning_rate)

# 训练轮数
epoch = 10
# 训练次数
total_train_step = 0
# 测试次数
total_test_step = 0

# 添加tensorboard
writer = SummaryWriter("./logs_train")

for i in range(epoch):
    print ("-----第 {} 轮训练开始-----".format(i + 1))
    # 训练步骤开始
    tudui.train()
    for data in train_dataloader:
        imgs, targets = data
        outputs = tudui(imgs)
        loss = loss_fn(outputs, targets)
        # 优化器优化模型
        optim.zero_grad()
        loss.backward()
        optim.step()
        total_train_step += 1 
        if total_train_step % 100 == 0:
            print("训练次数: {}, loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
        
    
    # 测试步骤开始
    tudui.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            outputs = tudui(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy
            
    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy*1.0/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy*1.0/test_data_size, total_test_step)
    total_test_step += 1
    
    torch.save(tudui, "./train_model/tudui_{}.pth".format(i + 1))
    print("模型已保存")

writer.close()

-----第 1 轮训练开始-----
训练次数: 100, loss: 2.293609142303467
训练次数: 200, loss: 2.274789571762085
训练次数: 300, loss: 2.246526002883911
训练次数: 400, loss: 2.1805224418640137
训练次数: 500, loss: 2.038132667541504
训练次数: 600, loss: 2.016104221343994
训练次数: 700, loss: 2.0101566314697266
整体测试集上的Loss: 315.8873119354248
整体测试集上的正确率: 0.26980000734329224
模型已保存
-----第 2 轮训练开始-----


/Users/wengyifan/opt/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Tudui. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


训练次数: 800, loss: 1.8818761110305786
训练次数: 900, loss: 1.8317431211471558
训练次数: 1000, loss: 1.885180115699768
训练次数: 1100, loss: 1.966273307800293
训练次数: 1200, loss: 1.7190194129943848
训练次数: 1300, loss: 1.6498054265975952
训练次数: 1400, loss: 1.7262297868728638
训练次数: 1500, loss: 1.7752535343170166
整体测试集上的Loss: 305.8274520635605
整体测试集上的正确率: 0.3012000024318695
模型已保存
-----第 3 轮训练开始-----
训练次数: 1600, loss: 1.7555162906646729
训练次数: 1700, loss: 1.670732855796814
训练次数: 1800, loss: 1.9443329572677612
训练次数: 1900, loss: 1.6910202503204346
训练次数: 2000, loss: 1.9491808414459229
训练次数: 2100, loss: 1.5387189388275146
训练次数: 2200, loss: 1.4959685802459717
训练次数: 2300, loss: 1.8203814029693604
整体测试集上的Loss: 262.2885853052139
整体测试集上的正确率: 0.39579999446868896
模型已保存
-----第 4 轮训练开始-----
训练次数: 2400, loss: 1.7225686311721802
训练次数: 2500, loss: 1.32748281955719
训练次数: 2600, loss: 1.5756664276123047
训练次数: 2700, loss: 1.6678221225738525
训练次数: 2800, loss: 1.4797271490097046
训练次数: 2900, loss: 1.6068066358566284
训练次数: 3000, loss